## **CARA collision probabiity 2D**
procedure
1. tle : space-track
2. dimension : DISCOSweb (esa)
3. CARA matlab code 이용, 확률 계산 </br>

참고 : TCA, HBR은 알고 있다는 전제하에 계산됨.

## **import**

In [1]:
from sgp4.api import Satrec, jday
import numpy as np
from datetime import timedelta, datetime
import requests
import json
import configparser
import requests

## **function**

In [2]:
#string을 datetime으로 변환
def str_to_datetime(date_str):
    return datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')

# TLE 데이터를 통해 특정 시점의 statevector 계산
def get_state_ECI(current_epoch, tle_line1, tle_line2):
    

    yr, mon, day = current_epoch.year, current_epoch.month, current_epoch.day
    hour, min, sec = current_epoch.hour, current_epoch.minute, current_epoch.second
    jd, fr = jday(yr, mon, day, hour, min, sec)

    satellite = Satrec.twoline2rv(tle_line1, tle_line2) 
    e, r, v = satellite.sgp4(jd,fr)
    position_np=np.array(r)
    velocity_np=np.array(v)
    return position_np, velocity_np

# spack-track TLE 주소를 넣고, 특정 feature 불러오기
def get_tle_element(feature, uri_base="https://www.space-track.org", request_login = "/ajaxauth/login", request_cmd_action = "/basicspacedata/query"):
    param=feature
    # set of satellite ids of objects included in TLE data
    SATNO_LIST = []

    # Use configparser package to pull in the ini file (pip install configparser)
    config = configparser.ConfigParser()
    config.read("./Login.ini")
    configUsr = config.get("configuration", "username")
    configPwd = config.get("configuration", "password")
    siteCred = {'identity': configUsr, 'password': configPwd}

    with requests.Session() as session:
        # run the session in a with block to force session to close if we exit

        # need to log in first. note that we get a 200 to say the web site got the data, not that we are logged in
        response = session.post(uri_base + request_login, data=siteCred)
        if response.status_code != 200:
            print("Error, POST fail on login")

        # this query picks up objects from the catalog. Note - a 401 failure shows you have bad credentials
        response = session.get(uri_base + request_cmd_action + request_find_objects)
        if response.status_code != 200:
            print("Error, GET fail on request")
            

        data = json.loads(response.text)
        for object in data:
            SATNO = object[f'{feature}']
            SATNO_LIST.append(SATNO)
    
        session.close()
        SATNO_LIST=np.array(SATNO_LIST).reshape(-1,1)
    return list(SATNO_LIST.flatten()) 

# ECI 좌표를 NTW로 변환
def calculate_trasformation_matrix_eci_to_ntw(true_position, true_velocity):
    t_axis = true_velocity/np.linalg.norm(true_velocity)

    rv_eci = np.cross(true_position, true_velocity)
    w_axis = rv_eci/np.linalg.norm(rv_eci)

    n_axis = np.cross(t_axis,w_axis)

    trasformation_matrix_ntw =(np.vstack((t_axis, n_axis, w_axis)))

    return trasformation_matrix_ntw

# ECI 좌표를 RSW로 변환
def calculate_trasformation_matrix_eci_to_rsw(true_position, true_velocity):
    r_axis = true_position/np.linalg.norm(true_position)
    rv_eci = np.cross(true_position, true_velocity)
    w_axis = rv_eci/np.linalg.norm(rv_eci)
    s_axis=np.cross(w_axis,r_axis)

    trasformation_matrix_rsw = (np.vstack((r_axis, s_axis, w_axis)))
    return trasformation_matrix_rsw
   
# 현재시점을 기준으로 과거 14일 tle 데이터로 공분산 계산
def calculate_covariance(satno):

    current_date = (datetime.now()) 

    start_date = str(current_date-timedelta(days=14))
    end_date = str(current_date) #종료일
    global request_find_objects
    request_find_objects = r= f"/class/tle/EPOCH/>{start_date}%2C<{end_date}/NORAD_CAT_ID/{satno}/orderby/EPOCH%20asc"

    epoch = get_tle_element("EPOCH")
    tle_line1 = get_tle_element("TLE_LINE1")
    tle_line2 = get_tle_element("TLE_LINE2")

    epoch_date = np.vectorize(str_to_datetime)(epoch) #datetime으로 변환.
    recent_epoch = epoch_date[-1] # 가장 최근시점을 기준으로 삼음
    N = epoch_date.size 
    position_residual_list =[]
    state_vector_list = []
    true_position, true_velocity = get_state_ECI(recent_epoch, tle_line1[-1], tle_line2[-1])
    tranformation_matrix_rsw = calculate_trasformation_matrix_eci_to_rsw(true_position, true_velocity)

    for i in range(N-1): 
        current_position, current_velocity = get_state_ECI(recent_epoch, tle_line1[i], tle_line2[i])
        position_residual = current_position-true_position
        velocity_residual = current_velocity-true_velocity

        state_matrix_rsw = tranformation_matrix_rsw @ np.transpose(np.vstack((position_residual,velocity_residual)))

        position_rsw ,velocity_rsw= state_matrix_rsw[:,0], state_matrix_rsw[:,1]
    
        state_vector = position_rsw.tolist()
        state_vector.extend(velocity_rsw.tolist())
        state_vector_list.append(state_vector)


        position_residual_list.append((tuple(position_rsw))) 
    covariance = np.transpose(state_vector_list - np.mean(state_vector_list,axis=0))@(state_vector_list - np.mean(state_vector_list,axis=0))/len(state_vector_list)
    recent_tle_line1 = tle_line1[-1]
    recent_tle_line2 = tle_line2[-1]
    return recent_tle_line1, recent_tle_line2, covariance

## **object 선정 후 공분산 계산**
STARLINK-5626 VS SL-8 DEB 

In [3]:
primary_id = 49437
secondary_id =  34542

primary_recent_tle_line1, primary_recent_tle_line2, primary_covariance = calculate_covariance(primary_id)
secondary_recent_tle_line1, secondary_recent_tle_line2,secondary_covariance = calculate_covariance(secondary_id)

## **matlab 엔진을 vscode로 연결**

In [4]:
import matlab.engine
eng=matlab.engine.start_matlab()

In [5]:
TCA = str_to_datetime("2024-01-30 05:07:58")

yr, mon, day = TCA.year, TCA.month, TCA.day
hour, min, sec = TCA.hour, TCA.minute, TCA.second
jd, fr = jday(yr, mon, day, hour, min, sec)

satellite1 = Satrec.twoline2rv(primary_recent_tle_line1, primary_recent_tle_line2) 
e1, r1, v1 = satellite1.sgp4(jd,fr)
satellite2 = Satrec.twoline2rv(secondary_recent_tle_line1, secondary_recent_tle_line2) 
e2, r2, v2 = satellite2.sgp4(jd,fr)

r1      = matlab.double(list(r1)) 
v1      = matlab.double(list(v1)) 
r2      = matlab.double(list(r2))
v2      = matlab.double(list(v2))
cov1    = matlab.double(list(primary_covariance))
cov2    = matlab.double(list(secondary_covariance))
HBR     = 2.22 
Tol     = 1e-09
HBRType = 'squareEquArea' # 영역설정 방법 : 직사각형으로 계산
empty_cov=matlab.double(np.zeros((3,3)))

## **확률 계산**

In [6]:
# matlab 함수 위치 설정
eng.addpath('C:/Users/VDRC/Documents/MATLAB/single_covariance_maximum_Pc/Main/ProbabilityOfCollisionCode',nargout=0)
print("Maximum Pc:",eng.FrisbeeMaxPc(r1,v1,cov1,r2,v2,cov2,HBR,Tol,HBRType)) 
print("Maximum Pc(empty):",eng.FrisbeeMaxPc(r1,v1,cov1,r2,v2,empty_cov,HBR,Tol,HBRType)) 
print("Foster Pc:",eng.Pc2D_Foster(r1,v1,cov1,r2,v2,cov2,HBR,Tol,HBRType)) 

Maximum Pc: 6.22001114708281e-06
Maximum Pc(empty): 6.22001114708281e-06
Foster Pc: 0.0


## **예제**

In [32]:
r1      = matlab.double([378.39559, 4305.721887, 5752.767554])
v1      = matlab.double([2.360800244, 5.580331936, -4.322349039])
r2      = matlab.double([374.5180598, 4307.560983, 5751.130418])
v2      = matlab.double([-5.388125081, -3.946827739, 3.322820358])
cov1    = matlab.double([[44.5757544811362,  81.6751751052616,  -67.8687662707124],
           [81.6751751052616,  158.453402956163,  -128.616921644857],
           [-67.8687662707124, -128.616921644858, 105.490542562701]])
cov2    = matlab.double([[2.31067077720423,  1.69905293875632  ,-1.4170164577661],
             [1.69905293875632,  1.24957388457206 , -1.04174164279599],
            [-1.4170164577661  ,-1.04174164279599, 0.869260558223714]])
HBR     = 0.020
Tol     = 1e-09
HBRType = 'squareEquArea'
#Pc      = 2.70601573490093e-05

In [33]:
# matlab 함수 위치 설정
eng.addpath('C:/Users/VDRC/Documents/MATLAB/single_covariance_maximum_Pc/Main/ProbabilityOfCollisionCode',nargout=0)
print("Maximum Pc:",eng.FrisbeeMaxPc(r1,v1,cov1,r2,v2,cov2,HBR,Tol,HBRType)) 
print("Maximum Pc(empty):",eng.FrisbeeMaxPc(r1,v1,cov1,r2,v2,empty_cov,HBR,Tol,HBRType)) 
print("Foster Pc:",eng.Pc2D_Foster(r1,v1,cov1,r2,v2,cov2,HBR,Tol,HBRType)) 

Maximum Pc: 0.0005538545701407567
Maximum Pc(empty): 3.143832451361428e-05
Foster Pc: 2.7060157349009515e-05
